In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

data = pd.read_csv('National_INFRA_2015-07-01.csv')
data.head()

,PATIENTID,PRIMPROCID,LTFCALC,SURGMONTH,SURGYEAR,SURGWEEKDAY,SSDI_MORTYEAR,VQI_MORTYEAR,ALL_MORTYEAR,DEAD,GENDER,RACE,ETHNICITY,HTIN,HTCM,WTLB,WTKG,AGE,AGECAT,LOS,LOS2,PROC_SURVIVALDAYS,LATESTLTFDAYS,PRIMARYINSURER,DCSTATUS,DCNOTHOME,TRANSFER,TRANSFERALL,SMOKING,ANYSMOKE,HTN,DIABETES,ANYDM,HBA1C,CAD,ANYCAD,CABG,PCI,ANYCBGPCI,CHF,ANYCHF,COPD,ANYCOPD,DIALYSIS,CREATININE,CREATININE_UMOL,YNCREAT,STRESS,ALLSTRESS,LIVING,PREOPAMBUL,ASACLASS,HGB,HEMO_L,HXBYPASS,HXCEACAS,HXANEURREP,HXPVI,HXMAJORAMP,PRE_OP_ASA,PRE_OP_P2Y12_ANTAGONIST,ASAP2Y12ANT,PRE_OP_STATIN,PRE_OP_BETA_BLOCKER,ANYBB,PRE_OP_ACE_I_ARB,PRE_OP_ANTICOAGULANT,INDICATIONRT,INDICATIONLT,PATHOLOGYRT,PATHOLOGYLT,PINBYPASSR,PINBYPASSL,PINPTASTENTR,PINPTASTENTL,PLEGBYPASSR,PLEGBYPASSL,PLEGPTASTENTR,PLEGPTASTENTL,PMAJAMPR,PMAJAMPL,PMINAMPR,PMINAMPL,PREOPABIR,PREOPABIL,PREOPTBIR,PREOPTBIL,PREDUPLEX,PREMRA,PRECTA,PREARTERIO,PREVEINMAP,URGENCY,ANESTHESIA,SIDE,SKINPREP,GRAFTORIGIN,GRAFTRECIPIENT,GRAFTVEINTYPE,VEINSEGS,PROSTHETIC,EBL,GROIN_INCISION,TOTALPROCTIME,VEIN_HARVEST_INCISION,VEIN_GRAFT_LOCATION,VEINCUFF,SEQGRAFT,CCPVI,CCENDAR,CCSUPRAINGUINAL,DOPPLER,DUPLEX,ARTERIOGRAM,WOUND,GRFTINFECTION,TRANSFUSION,YNTRANSFX,NEWMI,ANYPOMI,DYSRHYTHMIA,CHF2,RESPIRATORY,CHGRENAL,STROKE,IPSAMP,DCPATENCY,DCPATJUDGE,RTOR,ORBLEED,ORTHROMB,ORINFECTION,ORREVISION,POCX_INFRA,DISCHARGE_ASA,DISCHARGE_STATIN,DISCHARGE_P2Y12_ANTAGONIST,DISCHARGE_BETA_BLOCKER,DISCHARGE_ACE_I_ARB,DISCHARGE_ANTICOAGULANT,POSTOPABIR,POSTOPABIL,POSTOPTBIR,POSTOPTBIL,DCAMBUL,ABXSTART,ABXSTOP,ABXCEPH,SURGEONID,CENTERID,REGIONID,CONTACTBY,LTFSMOKING,LTFSTATUS,MORTCAUSE,LTF_ASA,LTF_P2Y12_ANTAGONIST,LTF_ANTICOAGULANT,LTF_BETA_BLOCKER,LTF_STATIN,LTF_ACE_I_ARB,LTFCURRAMB,LTFIPSSYMP,LTFCURPATENCY,LTFCURPATENCYDAYS,LTFPATJUDGE,LTFIPSABI,LTFIPSTBI,LTFINFECTION,LTFREVISION,LTFREVISIONDAYS,LTFTHROMBREVISION,LTFTHROMBREVISIONDAYS,LTFMAJAMP,LTFMAJAMPDAYS,STROKEORDEATH
0,6,13428,1,DEC,2009,6,NaN,NaN,NaN,0,2.0,5.0,0.0,64.0,163.0,211.0,96.0,61,4,5.0,5.0,1898,305.0,NaN,1.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,NaN,1.0,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.9,79.56,0.0,2.0,2.0,1.0,1.0,3.0,13.8,138.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.66,0.49,0.42,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,NaN,2.0,3.0,1.0,1.0,0.0,300.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,1.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,NaN,0.0,0.98,0.94,0.60,0.46,1.0,1.0,1.0,1.0,9,1,21.0,1.0,1.0,1.0,NaN,1.0,0.0,NaN,1.0,1.0,NaN,1.0,0.0,1.0,305.0,4.0,0.90,0.55,0.0,0.0,305.0,0.0,305.0,0.0,305.0,NaN
1,6,38228,0,NOV,2011,6,NaN,NaN,NaN,0,2.0,5.0,0.0,66.0,168.0,209.0,95.0,63,4,3.0,3.0,1205,53.0,NaN,1.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,6.9,1.0,1.0,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,1.0,88.40,0.0,0.0,0.0,1.0,1.0,3.0,11.0,110.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,2.0,1.0,NaN,NaN,2.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.4,0.95,0.25,0.70,1.0,0.0,0.0,1.0,1.0,1.0,3.0,1.0,1.0,5.0,4.0,5.0,1.0,0.0,250.0,0.0,330.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,1.0,NaN,0.0,0.90,0.87,0.64,0.54,1.0,1.0,1.0,1.0,9,1,21.0,1.0,1.0,1.0,NaN,1.0,0.0,NaN,1.0,1.0,NaN,1.0,0.0,1.0,53.0,4.0,0.92,0.64,0.0,0.0,53.0,0.0,53.0,0.0,53.0,0.0
2,8,6,0,FEB,2009,3,NaN,NaN,NaN,0,1.0,5.0,0.0,75.0,191.0,279.0,127.0,68,4,17.0,9.0,2202,9.0,NaN,3.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.3,114.92,0.0,0.0,0.0,1.0,1.0,3.0,12.3,123.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,NaN,NaN,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,NaN,NaN,NaN,1.0,0.0,1.0,0.0,1.0,1.0,3.0,1.0,NaN,5.0,7.0,1.0,1.0,0.0,150.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,2.0,1.0,0.0,1.0,8,1,21.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [65]:
df = data.copy()

df = data.dropna(axis=0, how='any', thresh=None, subset=['CHGRENAL'], inplace=False)
df['AKI'] = df['CHGRENAL'].apply(lambda x: 0 if x == 0 else 1)
df['SURGMONTH'] = df['SURGMONTH'].map({'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
                                       'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12})
df = df.drop([19931])
df = df.drop([7558])
df['bmi'] = df['WTKG']/(df['HTCM']/100)**2
df['LOS/Week'] = df['LOS']/7
df = df.drop(df[df['DIALYSIS'] == 2.0].index)
df['CURRENTSMOKING'] = df['SMOKING'].map({0: 0, 1: 0, 2:1})

df_demographics = df[[#General Features
                      'PATIENTID','PRIMPROCID',
                      #'LTFCALC', 'SURGMONTH', 'SURGYEAR', 'SURGWEEKDAY',
                      'GENDER', 'RACE', 'ETHNICITY', 'bmi', 'AGECAT',
                      #'LOS2', 'LOS', 'HTCM', 'WTKG', Converted features 
                      #'DEAD', 'PROC_SURVIVALDAYS', 'DCNOTHOME','LOS/Week', Unknown features before surgeries
                      #'LATESTLTFDAYS', Seems useless
                      'PRIMARYINSURER', 'TRANSFERALL',
                      #Demographic Features
                      'ANYSMOKE', 'HTN', 'ANYDM', 'ANYCAD', 'ANYCBGPCI', 'ANYCHF', 'ANYCOPD', 'DIALYSIS', 
                      'YNCREAT', 'ALLSTRESS', 'LIVING', 'PREOPAMBUL', 'ASACLASS', 'HGB', 'HXBYPASS', 'HXCEACAS', 
                      'HXANEURREP', 'HXPVI', 'HXMAJORAMP', 'ASAP2Y12ANT', 'PRE_OP_STATIN', 'ANYBB', 
                      'PRE_OP_ACE_I_ARB', 'PRE_OP_ANTICOAGULANT',
                      #'CURRENTSMOKING' instead of 'ANYSMOKING', 
                      #'CREATININE' instead of 'YNCREAT'
                      #Label
                      'AKI']]
df_demographics = df_demographics.dropna()

y_dempgraphics = df_demographics['AKI']
X_demographics = df_demographics.drop(columns = ['AKI'])
X_demographics_train, X_demographics_test, y_demographics_train, y_demographics_test =train_test_split(
                                        X_demographics, y_dempgraphics, test_size=0.33, random_state=42)
y_demographics_train.value_counts()/len(y_demographics_train)

/Users/Chen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/Chen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0    0.95449
1    0.04551
Name: AKI, dtype: float64

In [66]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
select_k_best_classifier = SelectKBest(chi2, k=22)
X_demographics_train_20 = pd.DataFrame(select_k_best_classifier.fit_transform(X_demographics_train, y_demographics_train))

In [67]:
mask = select_k_best_classifier.get_support()
selected_k_features = X_demographics_train.columns[mask]
selected_k_features

Index(['PATIENTID', 'PRIMPROCID', 'bmi', 'AGECAT', 'TRANSFERALL', 'ANYSMOKE',
       'ANYDM', 'ANYCAD', 'ANYCBGPCI', 'ANYCHF', 'DIALYSIS', 'YNCREAT',
       'ALLSTRESS', 'PREOPAMBUL', 'ASACLASS', 'HGB', 'HXBYPASS', 'HXCEACAS',
       'HXMAJORAMP', 'ANYBB', 'PRE_OP_ACE_I_ARB', 'PRE_OP_ANTICOAGULANT'],
      dtype='object')

In [68]:
X_demographics_train_20 = X_demographics_train[['bmi', 'AGECAT', 'TRANSFERALL', 'ANYSMOKE',
                                                'ANYDM', 'ANYCAD', 'ANYCBGPCI', 'ANYCHF', 'DIALYSIS', 'YNCREAT',
                                                'ALLSTRESS', 'PREOPAMBUL', 'ASACLASS', 'HGB', 'HXBYPASS', 'HXCEACAS',
                                                'HXMAJORAMP', 'ANYBB', 'PRE_OP_ACE_I_ARB', 'PRE_OP_ANTICOAGULANT']]
X_demographics_test_20 = X_demographics_test[['bmi', 'AGECAT', 'TRANSFERALL', 'ANYSMOKE',
                                              'ANYDM', 'ANYCAD', 'ANYCBGPCI', 'ANYCHF', 'DIALYSIS', 'YNCREAT',
                                              'ALLSTRESS', 'PREOPAMBUL', 'ASACLASS', 'HGB', 'HXBYPASS', 'HXCEACAS',
                                              'HXMAJORAMP', 'ANYBB', 'PRE_OP_ACE_I_ARB', 'PRE_OP_ANTICOAGULANT']]

## GEE

In [69]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [9]:
#X_demographics_train = X_demographics_train.drop(columns = ['PRIMPROCID'], axis = 1)
#X_demographics_test = X_demographics_test.drop(columns = ['PRIMPROCID'], axis = 1)

In [10]:
#X_demographics_train.head()

,PATIENTID,GENDER,RACE,ETHNICITY,bmi,AGECAT,PRIMARYINSURER,TRANSFERALL,ANYSMOKE,HTN,ANYDM,ANYCAD,ANYCBGPCI,ANYCHF,ANYCOPD,DIALYSIS,YNCREAT,ALLSTRESS,LIVING,PREOPAMBUL,ASACLASS,HGB,HXBYPASS,HXCEACAS,HXANEURREP,HXPVI,HXMAJORAMP,ASAP2Y12ANT,PRE_OP_STATIN,ANYBB,PRE_OP_ACE_I_ARB,PRE_OP_ANTICOAGULANT
15989,143828,2.0,5.0,0.0,30.536722,4,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,11.6,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
13324,112267,1.0,5.0,0.0,27.458654,6,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,10.4,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
18622,173883,2.0,5.0,0.0,36.730946,4,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,13.2,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
11368,94006,1.0,3.0,0.0,19.707964,5,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,11.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0
6569,44171,1.0,5.0,0.0,25.103878,3,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,3.0,13.3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [11]:
#X_demographics_train.shape

(9075, 32)

In [12]:
#patientID = X_demographics_train['PATIENTID']
#X_demographics_train = X_demographics_train = X_demographics_train.drop(columns = ['PATIENTID'], axis = 1)

In [13]:
#fam = sm.families.Binomial()
#ind = sm.cov_struct.Exchangeable()

In [14]:
#model = model = sm.GEE(np.array(y_demographics_train), np.array(X_demographics_train),
                       np.array(patientID), family=fam, cov_struct=ind)

In [15]:
#result = model.fit()

In [16]:
#print(result.summary())

                               GEE Regression Results                              
Dep. Variable:                           y   No. Observations:                 9075
Model:                                 GEE   No. clusters:                     8568
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                   5
Family:                           Binomial   Mean cluster size:                 1.1
Dependence structure:         Exchangeable   Num. iterations:                    11
Date:                     Thu, 15 Nov 2018   Scale:                           1.000
Covariance type:                    robust   Time:                         13:42:49
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.4191      0.118     -3.548      0.000      -0.651      -0.188
x2     

In [17]:
#patientID_test = X_demographics_test['PATIENTID']
#X_demographics_test = X_demographics_test.drop(columns = ['PATIENTID'], axis = 1)

In [21]:
#params = result.params

In [23]:
#predy = model.predict(params, X_demographics_test)

In [35]:
#predy[predy>0.2]

array([0.24543689, 0.20655036, 0.20224174, 0.21369028, 0.40842739,
       0.27934068, 0.23292621, 0.26105999, 0.2520824 , 0.34252047,
       0.30106138, 0.28892734, 0.46839832, 0.26479436, 0.46235687,
       0.44483333, 0.2808541 , 0.36493972, 0.24087275, 0.24718239,
       0.27931721, 0.31401742, 0.38973319, 0.29508811, 0.29614322,
       0.2005615 , 0.27920309, 0.22211032, 0.39242328, 0.30368847,
       0.22338503, 0.33734527, 0.37596062, 0.68251617, 0.26210311,
       0.38388807, 0.50790454, 0.23725504, 0.22708279, 0.47896262,
       0.20300772, 0.20482407, 0.24789473, 0.28728086, 0.23386643,
       0.33479747, 0.21618015, 0.37729747, 0.21673781, 0.23941612,
       0.25162592, 0.21156829, 0.32961594, 0.50583916, 0.25831701,
       0.28076255, 0.26449771, 0.21351816, 0.21438383, 0.3578286 ,
       0.31862214, 0.305679  , 0.25773537, 0.28356284, 0.50647355,
       0.25396156, 0.24311013, 0.28046307, 0.31124286, 0.29377696,
       0.37255149, 0.20022171, 0.28908127, 0.30623224, 0.25357

In [56]:
#pred_y = list(map(lambda x: 1 if x > 0.05 else 0, predy))

In [57]:
#print(classification_report(y_demographics_test, pred_y))

              precision    recall  f1-score   support

           0       0.97      0.73      0.83      4266
           1       0.09      0.59      0.16       204

   micro avg       0.72      0.72      0.72      4470
   macro avg       0.53      0.66      0.50      4470
weighted avg       0.93      0.72      0.80      4470



In [60]:
#X_demographics_train.head()

,GENDER,RACE,ETHNICITY,bmi,AGECAT,PRIMARYINSURER,TRANSFERALL,ANYSMOKE,HTN,ANYDM,ANYCAD,ANYCBGPCI,ANYCHF,ANYCOPD,DIALYSIS,YNCREAT,ALLSTRESS,LIVING,PREOPAMBUL,ASACLASS,HGB,HXBYPASS,HXCEACAS,HXANEURREP,HXPVI,HXMAJORAMP,ASAP2Y12ANT,PRE_OP_STATIN,ANYBB,PRE_OP_ACE_I_ARB,PRE_OP_ANTICOAGULANT
15989,2.0,5.0,0.0,30.536722,4,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,11.6,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
13324,1.0,5.0,0.0,27.458654,6,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,10.4,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
18622,2.0,5.0,0.0,36.730946,4,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,13.2,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
11368,1.0,3.0,0.0,19.707964,5,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,11.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0
6569,1.0,5.0,0.0,25.103878,3,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,3.0,13.3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [63]:
#X_demographics_train.head()

,GENDER,RACE,ETHNICITY,bmi,AGECAT,PRIMARYINSURER,TRANSFERALL,ANYSMOKE,HTN,ANYDM,ANYCAD,ANYCBGPCI,ANYCHF,ANYCOPD,DIALYSIS,YNCREAT,ALLSTRESS,LIVING,PREOPAMBUL,ASACLASS,HGB,HXBYPASS,HXCEACAS,HXANEURREP,HXPVI,HXMAJORAMP,ASAP2Y12ANT,PRE_OP_STATIN,ANYBB,PRE_OP_ACE_I_ARB,PRE_OP_ANTICOAGULANT
15989,2.0,5.0,0.0,30.536722,4,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,11.6,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
13324,1.0,5.0,0.0,27.458654,6,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,10.4,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
18622,2.0,5.0,0.0,36.730946,4,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,13.2,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
11368,1.0,3.0,0.0,19.707964,5,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,11.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0
6569,1.0,5.0,0.0,25.103878,3,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,3.0,13.3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


## VIF

In [58]:
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [72]:
X_demographics_train.head()

,PATIENTID,PRIMPROCID,GENDER,RACE,ETHNICITY,bmi,AGECAT,PRIMARYINSURER,TRANSFERALL,ANYSMOKE,HTN,ANYDM,ANYCAD,ANYCBGPCI,ANYCHF,ANYCOPD,DIALYSIS,YNCREAT,ALLSTRESS,LIVING,PREOPAMBUL,ASACLASS,HGB,HXBYPASS,HXCEACAS,HXANEURREP,HXPVI,HXMAJORAMP,ASAP2Y12ANT,PRE_OP_STATIN,ANYBB,PRE_OP_ACE_I_ARB,PRE_OP_ANTICOAGULANT
15989,143828,147378,2.0,5.0,0.0,30.536722,4,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,11.6,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
13324,112267,143738,1.0,5.0,0.0,27.458654,6,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,10.4,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
18622,173883,204042,2.0,5.0,0.0,36.730946,4,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,13.2,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
11368,94006,88392,1.0,3.0,0.0,19.707964,5,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,11.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0
6569,44171,176066,1.0,5.0,0.0,25.103878,3,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,3.0,13.3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [82]:
features = "+".join(X_demographics.drop(columns=['PATIENTID', 'PRIMPROCID'], axis = 1).columns)

In [83]:
X_demographics.columns

Index(['PATIENTID', 'PRIMPROCID', 'GENDER', 'RACE', 'ETHNICITY', 'bmi',
       'AGECAT', 'PRIMARYINSURER', 'TRANSFERALL', 'ANYSMOKE', 'HTN', 'ANYDM',
       'ANYCAD', 'ANYCBGPCI', 'ANYCHF', 'ANYCOPD', 'DIALYSIS', 'YNCREAT',
       'ALLSTRESS', 'LIVING', 'PREOPAMBUL', 'ASACLASS', 'HGB', 'HXBYPASS',
       'HXCEACAS', 'HXANEURREP', 'HXPVI', 'HXMAJORAMP', 'ASAP2Y12ANT',
       'PRE_OP_STATIN', 'ANYBB', 'PRE_OP_ACE_I_ARB', 'PRE_OP_ANTICOAGULANT'],
      dtype='object')

In [87]:
features

'GENDER+RACE+ETHNICITY+bmi+AGECAT+PRIMARYINSURER+TRANSFERALL+ANYSMOKE+HTN+ANYDM+ANYCAD+ANYCBGPCI+ANYCHF+ANYCOPD+DIALYSIS+YNCREAT+ALLSTRESS+LIVING+PREOPAMBUL+ASACLASS+HGB+HXBYPASS+HXCEACAS+HXANEURREP+HXPVI+HXMAJORAMP+ASAP2Y12ANT+PRE_OP_STATIN+ANYBB+PRE_OP_ACE_I_ARB+PRE_OP_ANTICOAGULANT'

In [88]:
from statsmodels.tools.tools import add_constant

In [89]:
X_demographics_train_constant = add_constant(X_demographics_train)

In [90]:
X_demographics_train_constant.head()

,const,PATIENTID,PRIMPROCID,GENDER,RACE,ETHNICITY,bmi,AGECAT,PRIMARYINSURER,TRANSFERALL,ANYSMOKE,HTN,ANYDM,ANYCAD,ANYCBGPCI,ANYCHF,ANYCOPD,DIALYSIS,YNCREAT,ALLSTRESS,LIVING,PREOPAMBUL,ASACLASS,HGB,HXBYPASS,HXCEACAS,HXANEURREP,HXPVI,HXMAJORAMP,ASAP2Y12ANT,PRE_OP_STATIN,ANYBB,PRE_OP_ACE_I_ARB,PRE_OP_ANTICOAGULANT
15989,1.0,143828,147378,2.0,5.0,0.0,30.536722,4,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,11.6,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
13324,1.0,112267,143738,1.0,5.0,0.0,27.458654,6,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,10.4,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
18622,1.0,173883,204042,2.0,5.0,0.0,36.730946,4,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,13.2,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
11368,1.0,94006,88392,1.0,3.0,0.0,19.707964,5,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,11.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0
6569,1.0,44171,176066,1.0,5.0,0.0,25.103878,3,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,3.0,13.3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [91]:
pd.Series([variance_inflation_factor(X_demographics_train_constant.values, i) 
           for i in range(X_demographics_train_constant.shape[1])], 
           index=X_demographics_train_constant.columns)

const                   187.758081
PATIENTID                 3.335263
PRIMPROCID                3.211392
GENDER                    1.072930
RACE                      1.095827
ETHNICITY                 1.091665
bmi                       1.127564
AGECAT                    1.318481
PRIMARYINSURER            1.155180
TRANSFERALL               1.027652
ANYSMOKE                  1.131204
HTN                       1.199386
ANYDM                     1.190156
ANYCAD                    1.438040
ANYCBGPCI                 1.496874
ANYCHF                    1.165531
ANYCOPD                   1.086115
DIALYSIS                  1.016182
YNCREAT                   1.038722
ALLSTRESS                 1.041070
LIVING                    1.056237
PREOPAMBUL                1.249640
ASACLASS                  1.148546
HGB                       1.008098
HXBYPASS                  1.116861
HXCEACAS                  1.051935
HXANEURREP                1.049751
HXPVI                     1.102956
HXMAJORAMP          

In [93]:
X_demographics_train_noID = X_demographics.drop(columns=['PATIENTID', 'PRIMPROCID'], axis = 1)
X_demographics_train_noID.head()

,GENDER,RACE,ETHNICITY,bmi,AGECAT,PRIMARYINSURER,TRANSFERALL,ANYSMOKE,HTN,ANYDM,ANYCAD,ANYCBGPCI,ANYCHF,ANYCOPD,DIALYSIS,YNCREAT,ALLSTRESS,LIVING,PREOPAMBUL,ASACLASS,HGB,HXBYPASS,HXCEACAS,HXANEURREP,HXPVI,HXMAJORAMP,ASAP2Y12ANT,PRE_OP_STATIN,ANYBB,PRE_OP_ACE_I_ARB,PRE_OP_ANTICOAGULANT
7,2.0,5.0,0.0,27.281746,5,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,11.7,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
37,2.0,5.0,0.0,24.092971,3,2.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0,15.4,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
75,1.0,5.0,0.0,26.277285,3,3.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,14.1,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
111,1.0,1.0,0.0,29.407596,4,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,13.6,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
112,1.0,1.0,0.0,29.407596,4,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,10.4,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0


In [94]:
pd.Series([variance_inflation_factor(X_demographics_train_noID.values, i) 
           for i in range(X_demographics_train_noID.shape[1])], 
           index=X_demographics_train_noID.columns)

GENDER                   8.969028
RACE                    29.719928
ETHNICITY                1.144576
bmi                     20.754197
AGECAT                  16.911671
PRIMARYINSURER           4.082661
TRANSFERALL              1.101585
ANYSMOKE                 6.550195
HTN                      9.369869
ANYDM                    2.185528
ANYCAD                   1.996463
ANYCBGPCI                2.229031
ANYCHF                   1.366073
ANYCOPD                  1.461933
DIALYSIS                 1.022214
YNCREAT                  1.110196
ALLSTRESS                1.442766
LIVING                  27.029836
PREOPAMBUL               7.757181
ASACLASS                38.390574
HGB                      1.732794
HXBYPASS                 1.560243
HXCEACAS                 1.151023
HXANEURREP               1.116502
HXPVI                    1.890759
HXMAJORAMP               1.173678
ASAP2Y12ANT              4.746937
PRE_OP_STATIN            3.536619
ANYBB                    3.045400
PRE_OP_ACE_I_A

## back to GEE and RF/GB

In [97]:
X_demographics_train.head()

,PATIENTID,PRIMPROCID,GENDER,RACE,ETHNICITY,bmi,AGECAT,PRIMARYINSURER,TRANSFERALL,ANYSMOKE,HTN,ANYDM,ANYCAD,ANYCBGPCI,ANYCHF,ANYCOPD,DIALYSIS,YNCREAT,ALLSTRESS,LIVING,PREOPAMBUL,ASACLASS,HGB,HXBYPASS,HXCEACAS,HXANEURREP,HXPVI,HXMAJORAMP,ASAP2Y12ANT,PRE_OP_STATIN,ANYBB,PRE_OP_ACE_I_ARB,PRE_OP_ANTICOAGULANT
15989,143828,147378,2.0,5.0,0.0,30.536722,4,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,11.6,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
13324,112267,143738,1.0,5.0,0.0,27.458654,6,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,10.4,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
18622,173883,204042,2.0,5.0,0.0,36.730946,4,3.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,13.2,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
11368,94006,88392,1.0,3.0,0.0,19.707964,5,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,4.0,11.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0
6569,44171,176066,1.0,5.0,0.0,25.103878,3,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,3.0,13.3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [117]:
X_demographics_train_vif = X_demographics_train.drop(columns=['PATIENTID', 'PRIMPROCID',
                                                              'ASACLASS', 'LIVING', 'RACE', 'bmi'], axis = 1)

In [118]:
fam = sm.families.Binomial()
ind = sm.cov_struct.Exchangeable()
model = model = sm.GEE(np.array(y_demographics_train), np.array(X_demographics_train_vif),
                       np.array(patientID), family=fam, cov_struct=ind)
result = model.fit()
print(result.summary())

                               GEE Regression Results                              
Dep. Variable:                           y   No. Observations:                 9075
Model:                                 GEE   No. clusters:                     8568
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:                   5
Family:                           Binomial   Mean cluster size:                 1.1
Dependence structure:         Exchangeable   Num. iterations:                    11
Date:                     Thu, 15 Nov 2018   Scale:                           1.000
Covariance type:                    robust   Time:                         18:14:21
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.4238      0.114     -3.710      0.000      -0.648      -0.200
x2     

In [119]:
params = result.params

In [121]:
X_demographics_test_vif = X_demographics_test.drop(columns=['PATIENTID', 'PRIMPROCID', 
                                                            'ASACLASS', 'LIVING', 'RACE', 'bmi'], axis = 1)

In [122]:
predy = model.predict(params, X_demographics_test_vif)

In [132]:
pred_y = list(map(lambda x: 1 if x > 0.122 else 0, predy))
print(classification_report(y_demographics_test, pred_y))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95      4266
           1       0.17      0.23      0.20       204

   micro avg       0.91      0.91      0.91      4470
   macro avg       0.57      0.59      0.58      4470
weighted avg       0.93      0.91      0.92      4470

